# Analysis of the SSVEP data

Let's load a couple libraries we'll use repeatedly.

In [ ]:
library(dplyr, warn.conflicts=FALSE)
library(ggplot2)

The dataframe we're about to load has already undergone fourier transform & projection to FSAverage cortical space, so we're dealing with vertex numbers and frequency bin amplitudes, across subject and pre/post intervention measurement times.  This assumes it's in the same folder as the notebook; adjust path as needed:

In [ ]:
proc.time() -> start
readr::read_csv("all_subjects-fsaverage-freq_domain-stc.csv") -> all_data
print(proc.time() - start)

Now let's add in some metadata about who is in which group:

In [ ]:
# "https://raw.githubusercontent.com/yeatmanlab/SSWEF/master/params" -> param_dir
file.path("..", "..", "params") -> param_dir

# load intervention groups
file.path(param_dir, "intervention_cohorts.yaml") -> intervention_file
yaml::read_yaml(intervention_file) -> intervention

# load pre-test cohorts
file.path(param_dir, "letter_knowledge_cohorts.yaml") -> pretest_file
yaml::read_yaml(pretest_file) -> pretest

Next we'll define a function that will compute noise (the mean of the 2 frequency bins below and 2 frequency bins above the current bin). We'll apply it within a `group_by` operation 

In [ ]:
# function to compute noise (mean of 2 bins above and below)
compute_noise <- function(x) {
    (lag(x, 2, 0) + lag(x, 1, 0) + lead(x, 1, 0) + lead(x, 2, 0)) / 4 -> noise
    return(noise)
}

Now we'll merge in the metadata, separate the `source` column into separate hemisphere and vertex number columns, and compute `noise` and `snr` all in one go.

In [ ]:
# this takes several minutes... might be worth it to split it into steps that assign back to `all_data`
# the comment lines are natural points at which to do this.
all_data %>%
    # merge in metadata
    mutate(subj_num=stringr::str_sub(subject, -4),
           pretest=ifelse(subj_num %in% pretest$LowerKnowledge, 'lower', 'upper'),
           intervention=ifelse(subj_num %in% intervention$LetterIntervention, 'letter', 'language')) %>%
    # separate the hemispheres
    tidyr::separate(source, c("hemi", "vertex"), sep="_") %>%
    mutate(hemi=stringr::str_to_lower(hemi),
           vertex=as.integer(vertex),
           source=NULL) %>%
    # compute noise and SNR
    group_by(subject, timepoint, hemi, vertex) %>%
    mutate(noise=compute_noise(value),
           snr=value/noise) %>%
    ungroup() ->
    all_data

Let's save ourselves the trouble of having to do that again, by saving the processed data, so we can skip ahead next time we run this.

In [ ]:
save(all_data, "processed_data.RData")  # binary format, more efficient
# readr::write_csv(all_data, "processed_data.csv")
# readr::read_csv("processed_data.csv") -> all_data

Now that our dataframe has all the variables we want, let's compute our (uncorrected) t-test of "signal bin" vs "mean of adjacent noise bins". This will yield a new dataframe with the t-test results. We'll write out the t-value table too, so we don't need to recompute it:

In [ ]:
# compute signal-vs-noise uncorrected t-test
all_data %>%
    filter(freq == 2, timepoint == "pre") %>%
    group_by(hemi, vertex) %>%
    do(broom::tidy(t.test(.$value, .$noise, paired=TRUE))) ->
    tvals

# readr::write_csv(tvals, "t-values.csv")
# readr::read_csv("t-values.csv") -> tvals

Now we merge the t-values back into the main dataframe, so we can use them to filter the data.

In [ ]:
# merge t-values back into main dataframe
all_data %>%
    left_join(tvals, by=c("hemi", "vertex")) ->
    all_data

OK, let's filter on t-values greater than 4, and use that as our ROI to compare the cohorts:

In [ ]:
all_data %>%
    filter(statistic >= 4, timepoint == "pre", freq == 2) %>%
    do(broom::tidy(t.test(.$value ~ .$pretest))) ->
    pretest_tvals

summary(pretest_tvals)

And now for the intervention effect:

In [ ]:
all_data %>%
    filter(statistic >= 4, freq == 2) %>%
    tidyr::pivot_wider(names_from=timepoint, values_from=value) %>%
    mutate(post_minus_pre=post - pre) %>%
    do(broom::tidy(t.test(.$post_minus_pre ~ .$intervention))) ->
    intervention_tvals

summary(pretest_tvals)